# Hidden Markov Model

In [1]:
import string

import torch
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
from importlib import reload

In [3]:
class HMM(torch.nn.Module):
    """
    Hidden Markov model with discrete states and observations.
    """
    def __init__(self, n_hidden, n_observed):
        """
        Initialize model with n_hidden states and n_observed states.
        """
        super(HMM, self).__init__()
        self.K = n_hidden
        self.V = n_observed
        self.prior_module = PriorModule(self.K)
        self.transition_module = TransitionModule(self.K)
        self.emission_module = EmissionModule(self.K, self.V)

        # enable cuda is it is available
        if torch.cuda.is_available():
            self.cuda()


### Prior probabilities

The prior probability vector $\boldsymbol{\alpha}$ is defined as
$$
\alpha[k] = P(z_0 = k) .
$$

Since $P(z_0)$ is probability distribution, it must sum to 1:
$$
\sum_{k = 0}^{K - 1} P(z_0 = k) = 1 .
$$

In order to preserve numerical precision, we store $\boldsymbol{\alpha}$ as real values with range from $-\infty$ to $\infty$. We name this variable `real_alpha`. Keeping `alpha` in real space also allows us to randomly initialize it using the standard normal distribution.

Accordingly, we must normalize `real_alpha` before returning it as the prior probability vector. In other words, we must normalize `real_alpha` such that
$$
\sum_{k = 0}^{K - 1} \alpha[k] = 1 .
$$

For this, we can use the `torch.nn.functional.softmax` function.

In [4]:
class PriorModule(torch.nn.Module):
    def __init__(self, K):
        """
        Create prior vector with random initial values.
        """
        super(PriorModule, self).__init__()
        self.K = K
        self.real_alpha = torch.nn.Parameter(torch.randn(K))

    def probs(self):
        """
        Return the prior probability vector.
        """
        return torch.nn.functional.softmax(
            ???
        )

### Transition probabilities

The transition probability matrix $\boldsymbol{B}$ is defined such that
$$
B[k, l] = P(z_{t+1} = l \mid z_{t} = k) .
$$
(Note: As per standard mathematical notation, we represent a matrix by an uppercase bold letter. The uppercase of $\beta$ is $B$.)

The `TransitionModule` will store the transition matrix as real values (i.e. their ranges are from $-\infty$ to $\infty$. It will normalize the transition matrix.

Therefore, this module needs to normalize the `real_Beta` in order to obtain proper probability values; that is, for each hidden state $i$,
$$
\sum_{l = 0}^{K-1} P(z_{t+1} = l \mid z_{t} = k) = 1 .
$$

### 

In [5]:
class TransitionModule(torch.nn.Module):
    def __init__(self, K):
        """
        Create transition matrix with K hidden states and random initial values.
        """
        super(TransitionModule, self).__init__()
        self.K = K
        # sample initial values from standard normal
        self.real_Beta = ???
        
    def probs(self):
        """
        Return the transition probability matrix.
        """
        return ???

### Emission probabilities

The emission probability matrix $\boldsymbol{\Gamma}$ is defined as 
$$
\Gamma[k, v] = P( x_t = v \mid z_t = k) .
$$

In [6]:
class EmissionModule(torch.nn.Module):
    def __init__(self, K, V):
        """
        Randomly initialize emission matirx with K hidden states and V observed states.
        """
        ???

    def probs(self):
        """
        Return the emission probability matrix.
        """
        return ???


### Sampling from the model

Once the parameters $(\boldsymbol{\alpha}, \boldsymbol{B}, \boldsymbol{\Gamma})$ are fully specified, we can sample from the HMM simply by following the model definition:

For $t = 0$,
$$
\begin{aligned}
z_0 \; &\sim \; \text{Categorical}( \boldsymbol{\alpha} ) , \\
x_0 \mid z_0 = k \; &\sim \; \text{Categorical}( \boldsymbol{\gamma}_k ) ,
\end{aligned}
$$
where $\boldsymbol{\gamma}_k$ is the $k$-th row of $\boldsymbol{\Gamma}$.

For $1 < t < T$,
$$
\begin{aligned}
z_t \mid z_{t-1} \; &\sim \; \text{Categorical}( \boldsymbol{\beta}_k ) , \\ 
x_t \mid z_t = k \; &\sim \; \text{Categorical}( \boldsymbol{\gamma}_k ) ,
\end{aligned}
$$
where $\boldsymbol{\beta}_k$ is the $k$-th row of $\boldsymbol{B}$.

We can draw a sample from a categorical distribution using `torch.distributions.categorical.Categorical`.

In [7]:

def sample(self, T=10):
    """
    Sample from the hidden Markov model for T time points.
    """

    from torch.distributions.categorical import Categorical

    # obtain normalized probabilities
    priors = self.prior_module.probs()
    transitions = self.transition_module.probs()
    emissions = self.emission_module.probs()

    # initialize state arrays
    z = np.zeros(T, dtype=np.int8)
    x = np.zeros(T, dtype=np.int8)
    
    # sample initial state
    z_t = Categorical(priors).sample().item()
    z[0] = z_t
    
    for t in range(0, T):
        # sample emission
        ???
        x[t] = x_t

        # sample transition
        ???
        z[t] = z_t

    return x, z

# add sampling method to HMM class
HMM.sample = sample

In [8]:
model = HMM(4, 6)

In [9]:
model.sample()

(array([1, 5, 5, 3, 3, 3, 1, 5, 1, 5], dtype=int8),
 array([2, 3, 2, 1, 2, 3, 2, 3, 2, 2], dtype=int8))

### Learning model parameters

You can hard-code HMM parameters to achieve the behaivour that 
you desire. However, we would typically want to learn the parameters by training our model on data.

The PyTorch allows us to minimize a loss function using stochastic gradient descent. Thanks to automatic differentiation, we will not have to analytically derive the gradient equations.

All we need to do is to specify and implement a loss function. A sensible objective function is the model evidence $p(\boldsymbol{x})$ for the observed data $\boldsymbol{x} = (x_0 \dots x_{T-1})$.

In HMM, the model evidence follows directly from its model specifiction:
$$
p(\boldsymbol{x}) 
= \sum_{\boldsymbol{z}} 
  p( \boldsymbol{x} \mid \boldsymbol{z} ) \; 
  p( \boldsymbol{z} )
= \sum_{\boldsymbol{z}} \; p(x_0 \mid z_0) \, p( z_0 ) \; 
  \prod_{t > 0} p( x_t \mid z_t ) \, p( z_t \mid z_{t-1} )
$$

We want to *maximize* the model evidence $p(\boldsymbol{x})$. To avoid numeric underflow, we can work in log scale such that we maximize $\log p(\boldsymbol{x}) $, which is mathematically maximizing $p(\boldsymbol{x})$. Since in PyTorch, we need to *minimize* a loss function, we can therefore define our loss function as
$$
L_{\theta}(\boldsymbol{x}) = - \log p_{\theta}(\boldsymbol{x}) ,
$$
where we use $\theta$ to represent the collection of all model parameters.

The model evidence $p(\boldsymbol{x})$ for HMM can be efficiently calculated using the dynamic programming algorithm, and in HMM, we call this algorithm the **forward algorithm**.

The key idea of the forward algorithm is that we can calculate the following probability efficiently:
$$
\begin{aligned}
a_{k, t} 
 &\triangleq p( x_0, \ldots, x_t, z_t = k) \\
 &= p( x_t \mid z_t = k) \;
   \sum_l \; 
    p( z_t = k \mid z_{t-1} = l ) \,
    p( x_0, \ldots, x_{t-1}, z_{t-1} = l ) \\
 &= \boldsymbol{\gamma}_k \;
   \sum_l \;
    \beta_{k, l} \, a_{l, t-1}
\end{aligned}
$$
And this recursive equation can be calculated efficient using dynamic programming.

After we calculate $a_{k,t}$, we can determine the model evidence by
$$
p(\boldsymbol{x}) = \sum_k a_{k, T}
$$

Again, in order to avoid numeric underflow, we work in log scale.


In [10]:
def forward_algorithm(self, x, T):
    """
    Compute log p(x) for each sample in a batch.
    
    x: IntTensor of shape (batch_size, T_max)
    T: IntTensor of shape (batch_size)
    
    x[i, :] contains a sample of input whose length is
    specified in T[i]. T_max is the maximum sample length 
    across all samples in a batch.
    """

    batch_size = x.shape[0]
    T_max = x.shape[1]
    
    log_priors = self.prior_module()
    
    log_a = torch.zeros(batch_size, T_max, self.K)
    log_a[:, 0, :] = self.emission_module(x[:,0]) + log_priors
    for t in range(1, T_max):
        log_a[:, t, :] = self.emission_module(x[:,t]) + \
            self.transition_module(log_a[:, t-1, :])
    
    # select the sum for the final time (T - 1)
    # (each sample x can have a different T).
    log_sums = log_a.logsumexp(dim=2)
    log_probs = torch.gather(log_sums, 1, T.view(-1,1) - 1)
    
    return log_probs

def prior_module_forward(self):
    """
    Return log prior probability vector.
    """
    return torch.nn.functional.log_softmax(self.real_alpha, dim=0)

def transition_module_forward(self, prev_log_a):
    """
    Return log a_{0:(K-1), t} using log a_{0:(K-1), t-1}
    
    prev_log_a : Tensor of shape (batch_size, K); column k is log_a{k, t-1}
    """
    log_probs = torch.nn.functional.log_softmax(self.real_Beta, dim=1)
    log_a = log_matmul(log_probs, prev_log_a.transpose(0,1)).transpose(0,1)
    return log_a
    
def emission_module_forward(self, x_t):
    """
    Return log emission probabilities p( x_t \mid z_t) for 
    observation x_t.
    """
    log_probs = torch.nn.functional.log_softmax(self.real_Gamma, dim=1)
    return log_probs[:, x_t].transpose(0,1)

def log_matmul(log_A, log_B):
	"""
    Compute log(A B) given log A and log B
 
	log_A: m x k
	log_B: k x n
	output: m x n matrix

    For C = A B, we have
	C_{i,j} = sum_k A_{i,k} x B_{k,j}

    Here, we compute log C by
	log C_{i,j} = logsumexp_k log_A_{i,k} + log_B_{k,j}
	"""
	m = log_A.shape[0]
	k = log_A.shape[1]
	n = log_B.shape[1]

	log_A_expanded = torch.reshape(log_A, (m,k,1))
	log_B_expanded = torch.reshape(log_B, (1,k,n))

	elementwise_sum = log_A_expanded + log_B_expanded
	out = torch.logsumexp(elementwise_sum, dim=1)

	return out

PriorModule.forward = prior_module_forward
TransitionModule.forward = transition_module_forward
EmissionModule.forward = emission_module_forward
HMM.forward = forward_algorithm


### Encoding and decoding

We will be working with text input, so we need to encode the data appropriately. For HMM, since we only use the encoded values for the purposing of indexing the emission and transition probability matrices, integer encoding is appropriate.

In [11]:
def encode(xs):
    character_set = string.ascii_lowercase
    return ???

def decode(ys, character_set=string.ascii_lowercase):
    character_set = string.ascii_lowercase
    return ???

In [12]:
encode("Hidden")

[7, 8, 3, 3, 4, 13]

In [13]:
decode(np.array([7, 8, 3, 3, 4, 13]))

'hidden'

In [14]:
decode(model.sample()[0])

'abefbfefdf'

### Running forward algorithm

With the forward algorithm and the encoder/decoder functions implemented, we are now ready to run the forward algorithm to calculate the log model evidence. This should output a floating value that is usually negative.

In [15]:
x = torch.stack( [torch.tensor(encode("cat"))] )
T = torch.tensor([3])

model = HMM(n_hidden=2, n_observed=26)

model.forward(x, T)

tensor([[-8.7857]], grad_fn=<GatherBackward0>)

Now that we have successfully implemented the forward pass in PyTorch, we can take advantage of its automatic differentiation capability to optimize our objective function by stochastic gradient descent. No mathematical derivation needed!

### Data preprocessing

In [16]:
!mkdir -p data
!wget -O data/words.txt https://github.com/dwyl/english-words/raw/master/words_alpha.txt

--2024-04-15 01:40:37--  https://github.com/dwyl/english-words/raw/master/words_alpha.txt
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt [following]
--2024-04-15 01:40:38--  https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4234917 (4.0M) [text/plain]
Saving to: ‘data/words.txt’

data/words.txt      100%[===================>]   4.04M  7.51MB/s    in 0.5s    

2024-04-15 01:40:38 (7.51 MB/s) - ‘data/words.txt’ saved [42

In [17]:
import utils.preprocess as pp
reload(pp)

in_fname = "data/words.txt"

with open(in_fname, "r") as f:
  words_all = [x.rstrip() for x in f.readlines()]

# use only words that begin with 'q' in order to reduce training time
words = []
for word in words_all:
    if word[0] == 'q':
        words.append(word)

# split data into training and testing
train_words, valid_words = train_test_split(words, test_size=0.1, random_state=1)
train_dataset = pp.TextDataset(train_words, encode)
valid_dataset = pp.TextDataset(valid_words, encode)

# construct character set
character_set = list(pp.Counter(("".join(words))).keys())
n_observed = len(character_set)

In [18]:
train_words[0:5]

['quixotically',
 'quadruplicity',
 'quadriplicated',
 'quadriplegia',
 'quincentenary']

In [19]:
valid_words[0:5]

['qual', 'quirite', 'quatorzes', 'quantizable', 'quebracho']

### Training

In [33]:
from utils import train
reload(train)

model = HMM(4, n_observed=n_observed)

# number of training passes through the data
n_epochs = 10
trainer = train.Trainer(model, learning_rate=0.01)

for epoch in range(n_epochs):
    print("========= Epoch %d of %d =========" % (epoch+1, n_epochs))
    train_loss = trainer.train(train_dataset, decode)
    valid_loss = trainer.test(valid_dataset, decode)

    print("========= Results: epoch %d of %d =========" % (epoch+1, n_epochs))
    print("train loss: %.2f| valid loss: %.2f\n" % (train_loss, valid_loss) )

========= Epoch 1 of 10 =========


  8%|█████▍                                                                | 1/13 [00:00<00:01,  8.80it/s]

loss: 31.002273559570312
ocynbnanpf
[1 1 0 2 0 2 0 2 0 2]
xkjxmxyxqb
[2 2 0 2 0 2 0 3 0 2]
ubbfuxxfmn
[2 2 0 2 0 0 0 0 0 2]
ixvxyoufow
[3 3 0 2 3 3 3 3 0 0]
sxxhppglvf
[3 0 2 0 2 2 2 3 3 0]


100%|█████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 49.08it/s]


loss: 32.7432746887207
qxklonveya
[3 2 2 0 0 3 1 2 1 1]
========= Results: epoch 1 of 10 =========
train loss: 31.45| valid loss: 32.59

========= Epoch 2 of 10 =========


...

loss: 27.388553619384766
qpqqclddaa
[0 3 3 3 1 1 1 0 0 2]
========= Results: epoch 9 of 10 =========
train loss: 25.67| valid loss: 27.00

========= Epoch 10 of 10 =========


 31%|█████████████████████▌                                                | 4/13 [00:00<00:00, 37.45it/s]

loss: 24.252410888671875
nuqapiaqxx
[3 3 3 0 0 0 3 3 0 0]
nenoslcinh
[2 0 3 0 0 2 1 0 3 3]
uqgaubaapu
[3 0 0 0 2 0 0 3 3 0]
iwdiquaaqi
[0 3 3 3 2 0 0 0 3 0]
uenqpidxfq
[3 0 0 2 0 2 0 0 3 1]


100%|█████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 52.40it/s]


loss: 26.315317153930664
exkaemepdu
[0 1 1 1 1 1 0 0 0 0]
========= Results: epoch 10 of 10 =========
train loss: 25.24| valid loss: 26.53



### Examining the trained model

We can examine the trained model by sampling a few words from it. We need to decode the output in order to read it.

In [34]:
for i in range(10):
    s = model.sample()
    print(
        "x:", decode(s[0]),
        ", z:", s[1]
    )

x: uqoniqqvia , z: [3 3 3 3 3 3 3 3 3 3]
x: iaqraranqq , z: [3 3 1 0 3 3 0 0 3 2]
x: oqnnaaregz , z: [3 0 0 1 1 1 1 0 0 0]
x: qaukroarle , z: [0 2 2 1 1 1 3 0 1 1]
x: suuqbqilnu , z: [3 3 3 3 3 3 2 0 3 3]
x: qawtcbraco , z: [3 3 3 3 3 3 3 3 3 3]
x: quenoqcqno , z: [3 2 0 0 3 3 3 0 3 3]
x: vqqxogqfhi , z: [0 0 2 1 3 3 3 3 3 3]
x: uqqfvtmuno , z: [3 3 3 0 0 0 3 3 3 0]
x: hiqqolrash , z: [3 3 3 3 1 1 0 0 3 3]


### Questions

1. How do you know whether the training is complete?
2. Is it better to have more or fewer hidden states in the HMM? Why?
3. How can you improve the model further?

### Bonus Questions

4. Train a HMM to learn the restriction endonuclease
   recognition sequences from [New England Biolabs][1].
   You will need to preprocess the data appropriately.

5. Implement the Viterbi algorithm.
   (Hint: it is very similar to the forward algorithm.)

[1]: https://www.neb.com/en/tools-and-resources/selection-charts/alphabetized-list-of-recognition-specificities